In [1]:
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
from data_utils import get_CIFAR10_data
import resource
import numpy as np
from numpy import float32

# reload(data_utils)
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [81]:
num_train = 5 #49000
num_epochs = 10
batch_size = 5
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

input_dim = 3*32*32
hidden_dim = 200
num_classes = 10
X = small_data['X_train'].astype(float32)
X = np.reshape(X, (X.shape[0], -1))
y = small_data['y_train']

X_val = small_data['X_val'].astype(float32)
X_val = np.reshape(X_val, (X_val.shape[0], -1))
y_val = small_data['y_val']

# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 3*32*32], name="x_placeholder")
y_ = tf.placeholder(tf.int64, shape=(None), name="y_placeholder")
keep_prob = tf.placeholder(tf.float32, name="keep_prob_placeholder")

W1 = tf.Variable(1e-3*tf.truncated_normal((input_dim, hidden_dim), stddev=0.1), name="W1")
W2 = tf.Variable((1e-3*np.random.randn(hidden_dim, num_classes)).astype(float32), name="W2")
b1 = tf.Variable(tf.zeros([hidden_dim]), name='b1')
b2 = tf.Variable(tf.zeros([num_classes]), name='b2')

L1_scores = tf.matmul(x, W1) + b1
L1 = tf.nn.relu(L1_scores)
L1_drop = tf.nn.dropout(L1, keep_prob)

# Readout layer
L2_scores = tf.matmul(L1_drop, W2) + b2

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(L2_scores, y_, name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

optimizer = tf.train.GradientDescentOptimizer(6e-4) #.001
# Create a variable to track the global step.
global_step = tf.Variable(0, name='global_step', trainable=False)

# Use the optimizer to apply the gradients that minimize the loss
# (and also increment the global step counter) as a single training step.
train_op = optimizer.minimize(loss, global_step=global_step)

# softmax = tf.nn.softmax(scores, name=None)
# loss = tf.reduce_mean(softmax, name='mean')
with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    for epoch in xrange(num_epochs):
        for i in xrange(num_train/batch_size):
            batch_xs = X[i*batch_size:i*batch_size+batch_size] # (batchsize, image_dim)
            batch_ys = y[i*batch_size:i*batch_size+batch_size] # (batchsize,)
            
            session.run(train_op, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5})
    #         print session.run(cross_entropy, feed_dict={x: batch_xs, y_: batch_ys})
            
        feed_train = {x: batch_xs, y_: batch_ys, keep_prob:1}
        feed_val = {x: X_val, y_: y_val, keep_prob:1}
        correct_prediction = tf.equal(tf.argmax(L2_scores, 1), y_)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_acc_result = accuracy.eval(feed_train)
        val_acc_result = accuracy.eval(feed_val)
        print("(Epoch: {e}/{et}) train acc {tacc}; val_acc: {vacc}".format(
                e=epoch+1,et=num_epochs, tacc=train_acc_result, vacc=val_acc_result))
        print session.run(loss, feed_dict=feed_train)


(Epoch: 1/10) train acc 1.0; val_acc: 0.144999995828
2.21674
(Epoch: 2/10) train acc 1.0; val_acc: 0.149000003934
2.06741
(Epoch: 3/10) train acc 1.0; val_acc: 0.15000000596
1.84415
(Epoch: 4/10) train acc 1.0; val_acc: 0.142000004649
1.53482
(Epoch: 5/10) train acc 1.0; val_acc: 0.136999994516
1.04663
(Epoch: 6/10) train acc 1.0; val_acc: 0.136000007391
0.697825
(Epoch: 7/10) train acc 1.0; val_acc: 0.141000002623
0.393693
(Epoch: 8/10) train acc 1.0; val_acc: 0.144999995828
0.276258
(Epoch: 9/10) train acc 1.0; val_acc: 0.141000002623
0.178238
(Epoch: 10/10) train acc 1.0; val_acc: 0.135000005364
0.157528


In [ ]:
num_epochs = 50
num_train = 5
batch_size = 5

num_classes = 10
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}
X = small_data['X_train'].astype(float32)
X = np.reshape(X, (X.shape[0], -1))
y = small_data['y_train']

input_dim = 3 * 32 * 32
num_filters = 32  # conv1 filters
num_filters2 = 64 # conv2 filters
C = 3
f_size = 5
fc_hidden_dim = 500

"""
Placeholders
"""
x_ = tf.placeholder(tf.float32, shape=[None, 3*32*32], name="x_placeholder")
x = tf.reshape(x_, [-1,32,32,3]) # double check if this reshape hasn't messed anything up:
y_ = tf.placeholder(tf.int64, shape=(None), name="y_placeholder")
keep_prob = tf.placeholder(tf.float32, name="keep_prob_placeholder")

"""
Conv layer 1
"""
W_conv1 = tf.Variable(1e-3*tf.truncated_normal((5, 5, C, num_filters), stddev=0.1), name="W_conv1" )
b_1 = tf.Variable(tf.zeros([num_filters]), name='b1')
h_conv1 = tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_1
h_relu = tf.nn.relu(h_conv1)
h_pool = tf.nn.max_pool(h_relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print h_pool


"""
Conv layer 2
"""
W_conv2 = tf.Variable(1e-3*tf.truncated_normal((5, 5, num_filters, num_filters2), stddev=0.1), name="W_conv2" )
b_2 = tf.Variable(tf.zeros([num_filters2]), name='b2')

print "conv 2"
h_conv_2 = tf.nn.conv2d(h_pool, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_2
h_relu_2 = tf.nn.relu(h_conv_2)
h_pool_2 = tf.nn.max_pool(h_relu_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print h_conv_2
print h_pool_2



"""
FC Layer
"""
# double check the 16 number
W_fc1 = tf.Variable(1e-3*tf.truncated_normal((16*16*num_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.zeros([fc_hidden_dim]), name="b_fc1")
h_pool_flat = tf.reshape(h_pool, [-1, 16*16*32])
print h_pool_flat
print W_fc1
# (10,8192) * (8192,500) = (10,500)
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
print h_fc1
"""
Output Layer
"""
W_fc2 = tf.Variable(1e-3*tf.truncated_normal((fc_hidden_dim,num_classes), stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.zeros([num_classes]), name="b_fc2")
scores_fc2 = (tf.matmul(h_fc1, W_fc2) + b_fc2)
print scores_fc2
# y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)
# print y_conv
# print tf.log(y_conv)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(scores_fc2, y_, name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
print cross_entropy
print loss


# optimizer = tf.train.GradientDescentOptimizer(1e-2)
# # Create a variable to track the global step.
# global_step = tf.Variable(0, name='global_step', trainable=False)
# train_op = optimizer.minimize(loss, global_step=global_step)

train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)


with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    
    for epoch in xrange(num_epochs):
        for i in xrange(num_train/batch_size):
            batch_xs = X[i*batch_size:i*batch_size+batch_size] # (batchsize, image_dim)
            batch_ys = y[i*batch_size:i*batch_size+batch_size] # (batchsize,)

            feed_train = {x_: batch_xs, y_: batch_ys}
            session.run(train_op, feed_dict=feed_train)
            
        print session.run(loss, feed_dict=feed_train)
        
        
        correct_prediction = tf.equal(tf.argmax(scores_fc2, 1), y_)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_acc_result = accuracy.eval(feed_train)
        print("(Epoch: {e}/{et}) train acc {tacc}".format(
                e=epoch+1,et=num_epochs, tacc=train_acc_result))
#         val_acc_result = accuracy.eval(feed_val)
    

Tensor("MaxPool_102:0", shape=(?, 16, 16, 32), dtype=float32)
conv 2
Tensor("add_247:0", shape=(?, 16, 16, 64), dtype=float32)
Tensor("MaxPool_103:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("Reshape_177:0", shape=(?, 8192), dtype=float32)
Tensor("Relu_174:0", shape=(?, 500), dtype=float32)
Tensor("add_249:0", shape=(?, 10), dtype=float32)
Tensor("xentropy_46:0", shape=(?,), dtype=float32)
Tensor("xentropy_mean_46:0", shape=(), dtype=float32)
2.30152
(Epoch: 1/50) train acc 0.40000000596
2.29499
(Epoch: 2/50) train acc 0.40000000596
2.2769
(Epoch: 3/50) train acc 0.40000000596
2.23929
(Epoch: 4/50) train acc 0.40000000596
2.174
(Epoch: 5/50) train acc 0.40000000596
2.07468
(Epoch: 6/50) train acc 0.40000000596
1.94022
(Epoch: 7/50) train acc 0.40000000596
1.77982
(Epoch: 8/50) train acc 0.40000000596
1.61668
(Epoch: 9/50) train acc 0.40000000596
1.48045
(Epoch: 10/50) train acc 0.40000000596
1.38433
(Epoch: 11/50) train acc 0.40000000596
1.32029
(Epoch: 12/50) train acc 0.4000000059